In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import re
import sys
import tarfile
import pandas

import numpy as np
from six.moves import urllib
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.misc import imsave

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#parameters 
input_width = 299
input_height = 299
input_depth = 3
input_mean = 128
input_std = 128
step = 0.01

In [3]:
def add_jpeg_decoding(input_width, input_height, input_depth, input_mean, input_std):
    """Adds operations that perform JPEG decoding and resizing to the graph..
    
    Args:
      input_width: Desired width of the image fed into the recognizer graph.
      input_height: Desired width of the image fed into the recognizer graph.
      input_depth: Desired channels of the image fed into the recognizer graph.
      input_mean: Pixel value that should be zero in the image for the graph.
      input_std: How much to divide the pixel values by before recognition.
    
    Returns:
      Tensors for the node to feed JPEG data into, and the output of the
        preprocessing steps.
    """
    jpeg_data = tf.placeholder(tf.string, name='DecodeJPGInput')
    decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
    decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
    decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
    resize_shape = tf.stack([input_height, input_width])
    resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
    resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                             resize_shape_as_int)
    offset_image = tf.subtract(resized_image, input_mean)
    mul_image = tf.multiply(offset_image, 1.0 / input_std)
    return jpeg_data, mul_image

In [4]:
def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile('classify_image_graph_def.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
    
    with tf.name_scope("new_last_layer"):
        weight = tf.get_variable('weight', [2048, 2])
        bias = tf.get_variable('bias', [2])
        bottle_neck = tf.placeholder(tf.float32, [None, 2048])
        ground_truth = tf.placeholder(tf.float32, [None, 2])
        
        prediction = tf.nn.softmax(tf.add(tf.matmul(bottle_neck, weight), bias), name='prediction')
        loss = tf.reduce_mean(tf.losses.log_loss(ground_truth, prediction), name='loss')
        
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(step)
        train_step = optimizer.minimize(loss)
        
    return bottle_neck, ground_truth, prediction, loss, train_step

In [5]:
def read_train_data(label_file, image_folder, valid_portion):
    df = pandas.read_csv(label_file).head(n=5000)
    print('Read %d records.' % len(df))
    df['image_path'] = [str(image_folder) + '/' + str(id) + '.jpg' for id in df['uid'].tolist()]
    valid_data = df[df.apply(lambda x: x['uid'] % 100 < valid_portion, axis=1)]
    train_data = df[~df.index.isin(valid_data.index)]
    return train_data, valid_data

In [6]:
def read_test_data(label_file, image_folder):
    df = pandas.read_csv(label_file).head(n=1000)
    print('Read %d records.' % len(df))
    df['image_path'] = [str(image_folder) + '/' + str(id) + '.jpg' for id in df['uid'].tolist()]
    return df

In [7]:
def df_2_dict(df_data):
    score = np.array(df_data['attractive_score'].tolist())
    return {
            'image_path': df_data['image_path'].tolist(),
            'score': np.column_stack((score, 1 - score))
    }

def generate_batch_data(train_data, batch_num):
    train_size = len(train_data)
    batch_size = int(train_size / batch_num)
    result = []
    perm = np.random.permutation(train_size)
    for batch in range(batch_num):
        indices = perm[batch * batch_size: (batch + 1) * batch_size]
        batch_data = train_data.iloc[indices]
        result.append(df_2_dict(batch_data))
    return result

In [8]:
def extra_feature(data, sess, mul_image, jpeg_data, feature_tensor, input_tensor):
    #parpare data
    feature = []
    score = []
    for idx in range(len(data['image_path'])):
        image_path = data['image_path'][idx]
        if tf.gfile.Exists(image_path):
            try:
                image_data = tf.gfile.FastGFile(image_path, 'rb').read()
                decoded_data = sess.run(mul_image, {jpeg_data: image_data})
                feature_score = sess.run(feature_tensor, {input_tensor: decoded_data})
                feature.append(np.squeeze(feature_score))
                score.append(data['score'][idx, :])
            except:
                pass
    feature = np.array(feature)
    score = np.array(score)
    return feature, score

In [9]:
def start_session():
    # Creates graph from saved GraphDef.
    bottle_neck, ground_truth, prediction, loss, train_step = create_graph()
    jpeg_data, mul_image = add_jpeg_decoding(input_width, input_height, input_depth, input_mean, input_std)
    saver = tf.train.Saver()
    sess = tf.Session()
    input_tensor = sess.graph.get_tensor_by_name('Mul:0')
    feature_tensor = sess.graph.get_tensor_by_name('pool_3/_reshape:0')
    return (bottle_neck, ground_truth, prediction, loss, train_step,
            jpeg_data, mul_image,
            input_tensor, feature_tensor,
            sess, saver)

In [10]:
def train_network(label_file, image_folder, valid_portion = 10, batch_num = 5, epoch = 10):
    train_data, valid_data = read_train_data(label_file, image_folder, valid_portion)
    valid_data = df_2_dict(valid_data)
        
    cache = {}

    #prepare valid data
    valid_feature, valid_score = extra_feature(valid_data, 
                                              sess, 
                                              mul_image, jpeg_data, 
                                              feature_tensor, input_tensor)

    # train the network
    sess.run(tf.global_variables_initializer())
    count = 0
    for ep in range(epoch):
        for batch_data in generate_batch_data(train_data, batch_num):
            feature, score = extra_feature(batch_data, 
                                           sess, 
                                           mul_image, jpeg_data, 
                                           feature_tensor, input_tensor)
            sess.run(train_step, {bottle_neck: feature, ground_truth: score})
            curr_loss = sess.run(loss, {bottle_neck: feature, ground_truth: score})
            print('step %d loss %f' % (count, curr_loss))
            count += 1

        valid_loss = sess.run(loss, {bottle_neck: valid_feature, ground_truth: valid_score})
        print('epoch %d loss %f'%(ep, valid_loss))
    saver.save(sess, "checkpoint/model.ckpt")

In [11]:
def predict(label_file, image_folder, test = True):
    # Creates graph from saved GraphDef.
    test_data = read_test_data(label_file, image_folder)
    test_data = df_2_dict(test_data)
    
    saver.restore(sess, "checkpoint/model.ckpt")

    test_feature, test_score = extra_feature(test_data, 
                                            sess, 
                                            mul_image, jpeg_data, 
                                            feature_tensor, input_tensor)
    if test:
        curr_loss = sess.run(loss, {bottle_neck: test_feature, ground_truth: test_score})
        print('Test loss: %f' % curr_loss)
    else:
        predict = sess.run(prediction, {bottle_neck: test_feature})
        return predict

In [ ]:
(bottle_neck, ground_truth, prediction, loss, train_step,
            jpeg_data, mul_image,
            input_tensor, feature_tensor,
            sess, saver) = start_session()
train_network('chunk2_uid_score.csv', 'chunk_2')

Read 5000 records.
step 0 loss 0.629265


In [ ]:
predict('chunk2_uid_score.csv', 'chunk_2')